In [3]:
# Install required libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [4]:
# Load the Fashion MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

100%|██████████| 26.4M/26.4M [00:03<00:00, 7.05MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 1.36MB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 6.44MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 106kB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [5]:
from torch.utils.data import random_split

# Define the split ratio for training and validation datasets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

# Split the training dataset into training and validation sets
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders for the training, validation, and test sets
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [6]:
# Define the neural network architecture
class FashionMNISTClassifier(nn.Module):
    def __init__(self):
        super(FashionMNISTClassifier, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hanna-jarlaczynska (glider-agh). Use `wandb login --relogin` to force relogin


True

In [9]:
epochs = 10
learning_rate = 0.001

In [10]:
run = wandb.init(
    # Set the project where this run will be logged
    project="fashion-mnist",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": learning_rate,
        "epochs": epochs,
    },
)

In [11]:
# Initialize the model, loss function, and optimizer
model = FashionMNISTClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)

In [12]:
# Training the model
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images, labels  # Ensure this correctly assigns (if using .to(), specify the device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            wandb.log({"train_loss": loss.item()})

        # Calculate validation loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images, val_labels
                val_outputs = model(val_images)
                val_loss += criterion(val_outputs, val_labels).item()

        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        # Log the average loss for the epoch
        wandb.log({"epoch": epoch + 1, "avg_train_loss": avg_train_loss, "avg_val_loss": avg_val_loss})
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
        
        model.train()  # Switch back to training mode

    print("Finished training")


In [15]:
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=epochs)

Epoch [1/10], Train Loss: 0.5032, Val Loss: 0.4214
Epoch [2/10], Train Loss: 0.3759, Val Loss: 0.3571
Epoch [3/10], Train Loss: 0.3346, Val Loss: 0.3436
Epoch [4/10], Train Loss: 0.3106, Val Loss: 0.3212
Epoch [5/10], Train Loss: 0.2868, Val Loss: 0.3427
Epoch [6/10], Train Loss: 0.2708, Val Loss: 0.3368
Epoch [7/10], Train Loss: 0.2552, Val Loss: 0.3254
Epoch [8/10], Train Loss: 0.2372, Val Loss: 0.3455
Epoch [9/10], Train Loss: 0.2250, Val Loss: 0.3293
Epoch [10/10], Train Loss: 0.2149, Val Loss: 0.3654
Finished training


In [16]:
# Save the model and log as an artifact
model_path = "model.pth"
torch.save(model.state_dict(), model_path)

artifact = wandb.Artifact("trained_model", type="model")
artifact.add_file(model_path)
wandb.log_artifact(artifact)

<Artifact trained_model>

In [17]:
artifact = wandb.Artifact("project_code", type="code")
artifact.add_file("wandb.ipynb")
wandb.log_artifact(artifact)

<Artifact project_code>

In [18]:
wandb.finish()

avg_train_loss,█▅▄▃▃▂▂▂▁▁
avg_val_loss,█▄▃▁▃▂▁▃▂▄
epoch,▁▂▃▃▄▅▆▆▇█
train_loss,▅▅▅▃▄▂█▄▄▄▇▄▅▂▇▄▅▃▄▄▄▄▃▂▃▃▄▅▃▄▃▂▄▂▄▃▂▂▂▁
avg_train_loss,0.21486
avg_val_loss,0.36538
epoch,10
train_loss,0.1926
